# KG Quality Evaluation Runner
Imports functions from `/src` and evaluates two CSV-based KGs.

In [1]:
import os, sys
import pandas as pd

# Add repo root to sys.path (similar to your pattern)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Try to load PROJECT_ROOT; fallback to cwd
try:
    from app_settings import PROJECT_ROOT
except Exception:
    PROJECT_ROOT = os.getcwd()

os.chdir(PROJECT_ROOT)
print(f"Changed working dir to {PROJECT_ROOT}")


Project root path: C:\wamp\www\KGs_for_Vertical_AI
046b1...
Environment variables loaded successfully
Changed working dir to C:\wamp\www\KGs_for_Vertical_AI


In [2]:
from src.kg_eval import evaluate_csv, format_report, interpret_kg_comparison_with_llm

## Configure CSVs & Columns

In [3]:

# CSV paths (based on your example)
RDB_CSV_PATH = "results/kgs/rdb_ontology_kg.csv"
TXT_CSV_PATH = "results/KGs/txt_ontology/txt_ontology_kg.csv"

# Edge columns
SOURCE_COL = "source"
TARGET_COL = "target"

DIRECTED = False
HEAVY_METRICS = True


## Evaluate the two KGs

In [4]:

rdb_eval = evaluate_csv(
    RDB_CSV_PATH,
    source_col=SOURCE_COL,
    target_col=TARGET_COL,
    directed=DIRECTED,
    heavy_metrics=HEAVY_METRICS
)
print(format_report(rdb_eval))


File: results/kgs/rdb_ontology_kg.csv
Nodes: 32 | Edges: 16 | Density: 0.0322581
Isolated nodes: 0 | Connected components: 16
Largest component ratio: 0.062
Clustering: 0.000

-- Quality Ratio --
Overall: 0.019
  Connectivity: 0.000
  Density: 0.000
  Component: 0.062


In [5]:

txt_eval = evaluate_csv(
    TXT_CSV_PATH,
    source_col=SOURCE_COL,
    target_col=TARGET_COL,
    directed=DIRECTED,
    heavy_metrics=HEAVY_METRICS
)
print(format_report(txt_eval))


File: results/KGs/txt_ontology/txt_ontology_kg.csv
Nodes: 62 | Edges: 31 | Density: 0.0163934
Isolated nodes: 0 | Connected components: 31
Largest component ratio: 0.032
Clustering: 0.000

-- Quality Ratio --
Overall: 0.262
  Connectivity: 0.361
  Density: 0.361
  Component: 0.032


## Side-by-side comparison

In [6]:

def to_flat_df(e):
    m = e['metrics'].copy()
    for k, v in e['quality_components'].items():
        m[f'QR_{k}'] = v
    m['QR_overall'] = e['quality_overall']
    m['path'] = e['path']
    return pd.DataFrame([m])

comp_df = pd.concat([to_flat_df(rdb_eval), to_flat_df(txt_eval)], ignore_index=True)
comp_df.set_index('path', inplace=True)
comp_df


,nodes,edges,density,isolated_nodes,connected_components,largest_component_ratio,avg_clustering,QR_ConnectivityScore,QR_DensityScore,QR_ComponentScore,QR_overall
path,,,,,,,,,,,
results/kgs/rdb_ontology_kg.csv,32,16,0.032258,0,16,0.062500,0.0,0.000000,0.000000,0.062500,0.018750
results/KGs/txt_ontology/txt_ontology_kg.csv,62,31,0.016393,0,31,0.032258,0.0,0.360656,0.360656,0.032258,0.262136


### Knowledge Graph Metrics Explained

- **nodes**: Total number of unique entities (nodes) in the graph.
- **edges**: Total number of relationships (edges) connecting nodes in the graph.
- **density**: Ratio of actual edges to all possible edges. Higher density means more connections relative to graph size.
- **isolated_nodes**: Number of nodes with no connections (degree zero). High values may indicate disconnected or unused entities.
- **connected_components**: Number of separate subgraphs in the KG. A lower number is better for retrieval, as it means the graph is more unified.
- **largest_component_ratio**: Fraction of nodes in the largest connected component. Values close to 1 mean most nodes are reachable from each other.
- **avg_clustering**: Measures how likely nodes are to form tightly-knit groups. Higher values indicate more local connectivity and context.

**Quality Component Scores**  
- **QR_ConnectivityScore**: A normalized score (0–1) reflecting how well-connected the graph is, based on density and structure.  
- **QR_DensityScore**: A normalized score (0–1) for edge density, indicating how richly the graph is populated with relationships.  
- **QR_ComponentScore**: A normalized score (0–1) for the largest component ratio, showing how unified the graph is.  
- **QR_overall**: The overall quality score (0–1), calculated as a weighted average of the three component scores—connectivity, density, and largest component ratio. The combined score summarizes the structural health of the knowledge graph for retrieval tasks.


### Interpreting KG Quality Reports with Azure OpenAI

We use Azure OpenAI (GPT-4.1 Nano) to generate expert interpretations of the knowledge graph quality reports. The LLM analyzes the structural metrics and quality scores, then provides insights, highlights strengths and weaknesses, and suggests improvements for each graph.

This step helps translate raw metrics into actionable recommendations for improving knowledge graph design and retrieval performance.

In [7]:
# Get LLM comparative interpretation for both KG reports
rdb_report = format_report(rdb_eval)
txt_report = format_report(txt_eval)
comparison_revision = interpret_kg_comparison_with_llm(rdb_report, txt_report)
print("KG Comparative LLM Interpretation:\n", comparison_revision)

KG Comparative LLM Interpretation:
 Winner: **Report 2 (txt_ontology_kg.csv)**

**Comparative Analysis:**

- **Size and Connectivity:**
  - Report 2 has nearly double the number of nodes (62 vs. 32) and edges (31 vs. 16), indicating a larger and more connected graph.
  - Both reports have no isolated nodes, but Report 2 has more connected components (31 vs. 16), suggesting it is more fragmented overall.

- **Density and Clustering:**
  - Report 1 has a slightly higher density (0.0323 vs. 0.0164), but both are very low, indicating sparse relationships.
  - Both have zero clustering coefficient, implying little local grouping or tightly-knit communities.

- **Largest Component Ratio:**
  - Report 1's largest component contains only 6.2% of nodes, whereas Report 2's largest component contains 3.2%. Despite being smaller, Report 2's larger size and higher overall scores suggest better structural health.

- **Quality Scores:**
  - Report 2's overall quality score (0.262) is significantly hi